Investigate ergosterol production in this model. 
Why doesnt it make a difference when erg5 and erg6 are knock outed? 

Try to knock out heterologous gene that is necessary to produce progesterone and test if it makes a difference. It should! 

If not too much work, test another model. 


# Knockout analysis

The heterologous pathway for production of progesterone has been implemented "iMM904_progesterone.xml". According to Yi-qi Jiang and Jian-ping Lin, it is necessary to knockout ERG5 and ERG6 in order to drive the flux toward the production of progesterone instad of the natural produced steriod ergosterol (Jiang, Yi-qi, and Jian-ping Lin 2022).

We want to investigate the influence it has on the cell to knock out ERG5 and ERG6 in our model.

In [114]:
# Load libraries
from cobra.io import read_sbml_model, write_sbml_model

In [115]:
# Load model
yeast_model = read_sbml_model('models/iMM904_progesterone.xml')
yeast_model

Name,iMM904_progesterone
Memory address,0x07fb7923a38e0
Number of metabolites,1234
Number of reactions,1586
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [116]:
# Progesterone is released from the cell
yeast_model.add_boundary(yeast_model.metabolites.get_by_id("progesterone_c"), type="demand")

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x7fb7923a3cd0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [117]:
yeast_model_knockout = yeast_model.copy()
yeast_model_knockout.genes.YMR015C.knock_out() # ERG5 knockout implemented
yeast_model_knockout.genes.YML008C.knock_out() # ERG6 knockout implemented

yeast_model_knockout_ERG4 = yeast_model.copy()
yeast_model_knockout_ERG4.genes.YGL012W.knock_out()

print('complete model: ', yeast_model.optimize()) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize()) # knockout of ERG5 and ERG6
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize()) 

complete model:  <Solution 0.288 at 0x7fb78fb0d190>
ERG5 and ERG6 knocked out:  <Solution 0.288 at 0x7fb78fb0d340>
ERG4 knocked out:  <Solution 0.000 at 0x7fb78fb0d2b0>


In [118]:
# Difference in progesterone production
yeast_model.objective = yeast_model.reactions.R02216
yeast_model_knockout.objective = yeast_model_knockout.reactions.R02216
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.R02216

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) 

complete model:  0.14285714285714285
ERG5 and ERG6 knocked out:  0.14285714285714285
ERG4 knocked out:  0.14285714285714285


In [119]:
# Ergosterol production
yeast_model.objective = yeast_model.reactions.ERGSTter
yeast_model_knockout.objective = yeast_model_knockout.reactions.ERGSTter
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.ERGSTter

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) # knockout

complete model:  0.18181818181818182
ERG5 and ERG6 knocked out:  0.18181818181818182
ERG4 knocked out:  0.0


In [112]:
# Ergosterol is naturally released from the cell
yeast_model.add_boundary(yeast_model.metabolites.get_by_id("ergst_c"), type="demand") # ergosterol

Reaction identifier,DM_ergst_c
Name,Ergosterol C28H44O demand
Memory address,0x7fb79d315e50
Stoichiometry,ergst_c --> Ergosterol C28H44O -->
GPR,
Lower bound,0
Upper bound,1000.0


In [102]:
# ERG5 and ERG6 influence on growth
print('complete model: ', yeast_model.optimize()) # no knockouts
yeast_model_knockout = yeast_model.copy()
yeast_model_knockout.genes.YMR015C.knock_out() # ERG5 knockout implemented
yeast_model_knockout.genes.YML008C.knock_out() # ERG6 knockout implemented
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize()) # knockout of ERG5 and ERG6 

complete model:  <Solution 0.288 at 0x7fb7937f2f70>
ERG5 and ERG6 knocked out:  <Solution 0.288 at 0x7fb79d360160>


In [103]:
# check ERG5 knockout
yeast_model_knockout.genes.YMR015C

Gene identifier,YMR015C
Name,ERG5
Memory address,0x7fb7937af730
Functional,False
In 2 reaction(s),"C22STDS, C22STDSx"


In [104]:
# check ERG6 knockout
yeast_model_knockout.genes.YML008C

Gene identifier,YML008C
Name,ERG6
Memory address,0x7fb7937ffb20
Functional,False
In 1 reaction(s),SAM24MT


In [8]:
# saving new model
outfilename = "models/iMM904_progesterone_ERG5_ERG6_knockout.xml"
yeast_model_knockout.id = outfilename.split("/")[-1].split(".")[0]
print(f"Saving to {outfilename}")
write_sbml_model(yeast_model_knockout, outfilename)

Saving to models/iMM904_progesterone_ERG5_ERG6_knockout.xml


Knocking out ERG5 and ERG6 have no influence on the growth of the strain which means that they can be implemented.

In [105]:
# Difference in progesterone production
yeast_model.objective = yeast_model.reactions.R02216
yeast_model_knockout.objective = yeast_model_knockout.reactions.R02216

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6 

complete model:  0.14285714285714285
ERG5 and ERG6 knocked out:  0.14285714285714285


Knocking out ERG5 and ERG6 does not have an influence on the production of progesterone as expected (Jiang, Yi-qi, and Jian-ping Lin 2022). Therefore, we will as a starting point use "iMM904_progesterone" as our model for further cell factory engineering studies.

In [67]:
yeast_model.optimize()

,fluxes,reduced_costs
CITtcp,0.000000,0.000000
13BGH,0.000000,0.000000
13BGHe,0.000000,0.000000
13GS,0.000000,0.000000
16GS,0.000000,0.000000
...,...,...
R07215,0.142857,0.000000
R04804,0.000000,0.000000
SK_methylpentanal_c,0.142857,0.000000
DM_progesterone_c,0.142857,0.000000


In [68]:
yeast_model_knockout.optimize()

,fluxes,reduced_costs
CITtcp,0.000000,0.000000
13BGH,0.000000,0.000000
13BGHe,0.000000,0.000000
13GS,0.000000,0.000000
16GS,0.000000,0.000000
...,...,...
R07215,0.142857,0.000000
R04804,0.000000,0.000000
SK_methylpentanal_c,0.142857,0.000000
DM_progesterone_c,0.142857,0.000000


In [69]:
# Investigate if the knockout of ERG5 and ERG6 influences other reactions
fluxes = yeast_model.optimize().fluxes
fluxes_knockout = yeast_model_knockout.optimize().fluxes
for i, (flux, flux_knockout) in enumerate(zip(fluxes, fluxes_knockout)):
    if flux != flux_knockout:
        print(f'Reaction {fluxes.index[i]}: flux: {flux} is different from knockout model flux: {flux_knockout}')

Reaction ACALDtm: flux: -5.296703296703296 is different from knockout model flux: -2.5714285714285694
Reaction ACLSm: flux: 0.6227106227106227 is different from knockout model flux: 0.0
Reaction ACS: flux: 0.6227106227106227 is different from knockout model flux: 0.0
Reaction ACt2r: flux: -1.3882783882783958 is different from knockout model flux: -0.1428571428571347
Reaction ACtm: flux: -0.6227106227106227 is different from knockout model flux: 0.0
Reaction 3C3HMPt: flux: 0.6227106227106227 is different from knockout model flux: 0.0
Reaction ACtr: flux: 1.3882783882783958 is different from knockout model flux: 0.1428571428571347
Reaction ALCD2ir: flux: 10.83882783882784 is different from knockout model flux: 14.80952380952381
Reaction ALCD2irm: flux: 4.673992673992673 is different from knockout model flux: 2.5714285714285694
Reaction ADK1: flux: 2.725274725274725 is different from knockout model flux: 0.8571428571428577
Reaction ADK3: flux: -2.102564102564102 is different from knockout

In [70]:
# Check if progesterone is produced in yeast model
yeast_model.objective = yeast_model.reactions.R02216
print('complete model: ', yeast_model.optimize().objective_value) # no knockouts

complete model:  0.14285714285714285


In [71]:
# Check if progesterone is produced when knocking out reaction
yeast_model_knockout_3betaHSD = yeast_model.copy()
yeast_model_knockout.genes.HSD3B.knock_out()
yeast_model_knockout_3betaHSD.objective = yeast_model_knockout_3betaHSD.reactions.R02216
print('Knocked out: ', yeast_model_knockout.optimize().objective_value)

Knocked out:  0.0


In [72]:
for i in yeast_model.metabolites.zymst_c.reactions:
    print(i)

ZYMSTAT_SC: 0.655 hdcoa_c + 0.01 hexccoa_c + 0.27 odecoa_c + 0.02 pmtcoa_c + 0.03 stcoa_c + 0.015 tdcoa_c + 0.01 zymst_c --> coa_c + 0.01 zymstest_SC_c
ZYMSTt: zymst_e <=> zymst_c
CHLSTI: amet_c + o2_c + zymst_c --> ahcys_c + ergtetrol_c + 2.0 h2o_c + h_c
R07498: h_c + nadph_c + zymst_c <=> cholesta8en3betaol_c + nadp_c
C3STKR2: h_c + nadph_c + zym_int2_c --> nadp_c + zymst_c
R04804: zymst_c --> cholesta724dien3betaol_c
SAM24MT: amet_c + zymst_c --> ahcys_c + fecost_c + h_c
ZYMSTESTH_SC: h2o_c + 0.01 zymstest_SC_c --> h_c + 0.02 hdca_c + 0.655 hdcea_c + 0.01 hexc_c + 0.03 ocdca_c + 0.27 ocdcea_c + 0.015 ttdca_c + 0.01 zymst_c
BIOMASS_SC5_notrace: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp_c + 0.0036 dtmp_c + 0.0007 ergst_c + 0.1054 gln__L_c + 0.3018 glu__L_c + 0.2904 gly_c + 0.5185 glycogen_c + 0.046 gmp_c + 59.276 h2o_c + 0.0663 his

In [42]:
yeast_model.genes.YML008C

Gene identifier,YML008C
Name,ERG6
Memory address,0x7fb7be9d3580
Functional,True
In 1 reaction(s),SAM24MT


In [35]:
for i in yeast_model.metabolites.fecost_c.reactions:
    print(i)

C8STI: fecost_c --> epist_c
FECOSTAT_SC: 0.01 fecost_c + 0.655 hdcoa_c + 0.01 hexccoa_c + 0.27 odecoa_c + 0.02 pmtcoa_c + 0.03 stcoa_c + 0.015 tdcoa_c --> coa_c + 0.01 fecostest_SC_c
FECOSTt: fecost_e <=> fecost_c
SAM24MT: amet_c + zymst_c --> ahcys_c + fecost_c + h_c
FECOSTESTH_SC: 0.01 fecostest_SC_c + h2o_c --> 0.01 fecost_c + h_c + 0.02 hdca_c + 0.655 hdcea_c + 0.01 hexc_c + 0.03 ocdca_c + 0.27 ocdcea_c + 0.015 ttdca_c


In [36]:
yeast_model.genes.YMR202W

Gene identifier,YMR202W
Name,ERG2
Memory address,0x7fb7bea99970
Functional,True
In 3 reaction(s),"R04804, C8STI, R03353"


In [37]:
for i in yeast_model.metabolites.epist_c.reactions:
    print(i)

C8STI: fecost_c --> epist_c
EPISTt: epist_e <=> epist_c
EPISTAT_SC: 0.01 epist_c + 0.655 hdcoa_c + 0.01 hexccoa_c + 0.27 odecoa_c + 0.02 pmtcoa_c + 0.03 stcoa_c + 0.015 tdcoa_c --> coa_c + 0.01 epistest_SC_c
C5STDS: epist_c + h_c + nadph_c + o2_c --> ergtrol_c + 2.0 h2o_c + nadp_c
EPISTESTH_SC: 0.01 epistest_SC_c + h2o_c --> 0.01 epist_c + h_c + 0.02 hdca_c + 0.655 hdcea_c + 0.01 hexc_c + 0.03 ocdca_c + 0.27 ocdcea_c + 0.015 ttdca_c


In [38]:
yeast_model.genes.YLR056W

Gene identifier,YLR056W
Name,ERG3
Memory address,0x7fb7bea998b0
Functional,True
In 2 reaction(s),"R07215, C5STDS"


In [43]:
for i in yeast_model.metabolites.ergtrol_c.reactions:
    print(i)

C22STDSx: ergtrol_c + h_c + nadh_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nad_c
C22STDS: ergtrol_c + h_c + nadph_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nadp_c
C5STDS: epist_c + h_c + nadph_c + o2_c --> ergtrol_c + 2.0 h2o_c + nadp_c


In [44]:
yeast_model.genes.YMR015C

Gene identifier,YMR015C
Name,ERG5
Memory address,0x7fb7bea99550
Functional,True
In 2 reaction(s),"C22STDSx, C22STDS"


In [48]:
yeast_model.metabolites.ergtetrol_c

Metabolite identifier,ergtetrol_c
Name,Ergosta 5 7 22 24 28 tetraen 3beta ol C28H42O
Memory address,0x7fb7bed40400
Formula,C28H42O
Compartment,c
In 4 reaction(s),"C22STDSx, C22STDS, ERGTETROLter, CHLSTI"


In [49]:
for i in yeast_model.metabolites.ergtetrol_c.reactions:
    print(i)

C22STDSx: ergtrol_c + h_c + nadh_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nad_c
C22STDS: ergtrol_c + h_c + nadph_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nadp_c
ERGTETROLter: ergtetrol_c <=> ergtetrol_r
CHLSTI: amet_c + o2_c + zymst_c --> ahcys_c + ergtetrol_c + 2.0 h2o_c + h_c


In [54]:
for i in yeast_model.metabolites.ergst_r.reactions:
    print(i)

ERGSTter: ergst_r <=> ergst_c
C24STRer: ergtetrol_r + h_r + nadph_r --> ergst_r + nadp_r


In [53]:
for i in yeast_model.metabolites.ergtetrol_r.reactions:
    print(i)

ERGTETROLter: ergtetrol_c <=> ergtetrol_r
C24STRer: ergtetrol_r + h_r + nadph_r --> ergst_r + nadp_r


In [50]:
for i in yeast_model.metabolites.ergst_c.reactions:
    print(i)

ERGSTGLCT: ergst_c + udpg_c --> ergst3glc_c + h_c + udp_c
ERGSTter: ergst_r <=> ergst_c
BIOMASS_SC5_notrace: 1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024 dgmp_c + 0.0036 dtmp_c + 0.0007 ergst_c + 0.1054 gln__L_c + 0.3018 glu__L_c + 0.2904 gly_c + 0.5185 glycogen_c + 0.046 gmp_c + 59.276 h2o_c + 0.0663 his__L_c + 0.1927 ile__L_c + 0.2964 leu__L_c + 0.2862 lys__L_c + 0.8079 mannan_c + 0.0507 met__L_c + 6e-06 pa_SC_c + 6e-05 pc_SC_c + 4.5e-05 pe_SC_c + 0.1339 phe__L_c + 0.1647 pro__L_c + 1.7e-05 ps_SC_c + 5.3e-05 ptd1ino_SC_c + 0.00099 ribflv_c + 0.1854 ser__L_c + 0.02 so4_c + 0.1914 thr__L_c + 0.0234 tre_c + 6.6e-05 triglyc_SC_c + 0.0284 trp__L_c + 0.102 tyr__L_c + 0.0599 ump_c + 0.2646 val__L_c + 0.0015 zymst_c --> 59.276 adp_c + 58.70001 h_c + 59.305 pi_c
ERGSTESTH_SC: 0.01 ergstest_SC_c + h2o_c --> 0.01 ergst_c + h_c + 0.02 hdca_c + 0.655 

In [52]:
for i in yeast_model.genes.YGL012W.reactions:
    print(i)

C24STRer: ergtetrol_r + h_r + nadph_r --> ergst_r + nadp_r


In [31]:
yeast_model.metabolites.fecost_c

Metabolite identifier,fecost_c
Name,Fecosterol C28H46O
Memory address,0x7fb7bedc0040
Formula,C28H46O
Compartment,c
In 5 reaction(s),"C8STI, FECOSTAT_SC, FECOSTt, SAM24MT, FECOSTESTH_SC"


In [80]:
# Check fluxes
yeast_model.objective = yeast_model.reactions.C22STDS
yeast_model_knockout.objective = yeast_model_knockout.reactions.C22STDS
print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6

complete model:  0.16666666666666666
ERG5 and ERG6 knocked out:  0.0


In [90]:
yeast_model.objective = yeast_model.reactions.ERGTETROLter
yeast_model_knockout.objective = yeast_model_knockout.reactions.ERGTETROLter
print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6

complete model:  0.18181818181818182
ERG5 and ERG6 knocked out:  0.18181818181818182


In [92]:
yeast_model.objective = yeast_model.reactions.ERGSTter
yeast_model_knockout.objective = yeast_model_knockout.reactions.ERGSTter
print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6

complete model:  0.18181818181818182
ERG5 and ERG6 knocked out:  0.18181818181818182


In [85]:
yeast_model.reactions.C22STDS

Reaction identifier,C22STDS
Name,C 22 sterol desaturase NADP
Memory address,0x7fb7adab8df0
Stoichiometry,ergtrol_c + h_c + nadph_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nadp_c Ergosta 5 7 24 28 trienol C28H44O + H+ + Nicotinamide adenine dinucleotide phosphate - reduced + O2 O2 --> Ergosta 5 7 22 24 28 tetraen 3beta ol C28H42O + 2.0 H2O H2O + Nicotinamide adenine...
GPR,YMR015C and YHR042W
Lower bound,0.0
Upper bound,999999.0


In [86]:
yeast_model.reactions.ERGTETROLter

Reaction identifier,ERGTETROLter
Name,Ergosta 5 6 22 24 28 tetraen 3beta ol endoplamic reticular transport
Memory address,0x7fb7ad8bd730
Stoichiometry,ergtetrol_c <=> ergtetrol_r Ergosta 5 7 22 24 28 tetraen 3beta ol C28H42O <=> Ergosta 5 7 22 24 28 tetraen 3beta ol C28H42O
GPR,
Lower bound,-999999.0
Upper bound,999999.0


In [95]:
yeast_model.reactions.ERGSTter

Reaction identifier,ERGSTter
Name,Ergosterol endoplasmic reticular transport
Memory address,0x7fb7ad8bdc10
Stoichiometry,ergst_r <=> ergst_c Ergosterol C28H44O <=> Ergosterol C28H44O
GPR,
Lower bound,-999999.0
Upper bound,999999.0


In [107]:
yeast_model_knockout_ERG4 = yeast_model.copy()
yeast_model_knockout_ERG4.genes.YGL012W.knock_out() # ERG4 knockout implemented

yeast_model.objective = yeast_model.reactions.ERGTETROLter
yeast_model_knockout.objective = yeast_model_knockout.reactions.ERGTETROLter
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.ERGTETROLter
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout
print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) # knockout

ERG5 and ERG6 knocked out:  0.18181818181818182
complete model:  0.18181818181818182
ERG4 knocked out:  0.0


In [93]:
yeast_model.objective = yeast_model.reactions.C24STRer
yeast_model_knockout.objective = yeast_model_knockout.reactions.C24STRer
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.C24STRer

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) # knockout

complete model:  0.18181818181818182
ERG5 and ERG6 knocked out:  0.18181818181818182
ERG4 knocked out:  0.0


In [94]:
# Ergosterol production
yeast_model.objective = yeast_model.reactions.ERGSTter
yeast_model_knockout.objective = yeast_model_knockout.reactions.ERGSTter
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.ERGSTter

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) # knockout

complete model:  0.18181818181818182
ERG5 and ERG6 knocked out:  0.18181818181818182
ERG4 knocked out:  0.0


In [108]:
# Difference in progesterone production
yeast_model.objective = yeast_model.reactions.R02216
yeast_model_knockout.objective = yeast_model_knockout.reactions.R02216
yeast_model_knockout_ERG4.objective = yeast_model_knockout_ERG4.reactions.R02216

print('complete model: ', yeast_model.optimize().objective_value) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize().objective_value) # knockout of ERG5 and ERG6
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize().objective_value) 

complete model:  0.14285714285714285
ERG5 and ERG6 knocked out:  0.14285714285714285
ERG4 knocked out:  0.14285714285714285


In [113]:
yeast_model_knockout = yeast_model.copy()
yeast_model_knockout.genes.YMR015C.knock_out() # ERG5 knockout implemented
yeast_model_knockout.genes.YML008C.knock_out() # ERG6 knockout implemented

yeast_model_knockout_ERG4 = yeast_model.copy()
yeast_model_knockout_ERG4.genes.YGL012W.knock_out()

print('complete model: ', yeast_model.optimize()) # no knockouts
print('ERG5 and ERG6 knocked out: ', yeast_model_knockout.optimize()) # knockout of ERG5 and ERG6
print('ERG4 knocked out: ', yeast_model_knockout_ERG4.optimize()) 

complete model:  <Solution 0.288 at 0x7fb7916114c0>
ERG5 and ERG6 knocked out:  <Solution 0.288 at 0x7fb791611310>
ERG4 knocked out:  <Solution 0.000 at 0x7fb7916113a0>


In [ ]:
yeast_model

In [43]:
print(yeast_model.optimize().fluxes)
for i in yeast_model.optimize().fluxes:
    print(i)

CITtcp                 0.000000
13BGH                  0.000000
13BGHe                 0.000000
13GS                   0.000000
16GS                   0.000000
                         ...   
R07215                 0.142857
R04804                 0.000000
SK_methylpentanal_c    0.142857
DM_progesterone_c      0.142857
DM_ergst_c             0.000000
Name: fluxes, Length: 1588, dtype: float64
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-5.296703296703296
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.6227106227106227
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.6227106227106227
0.0
0.0
-1.3882783882783958
0.0
0.0
0.0
0.0
-0.6227106227106227
0.6227106227106227
0.0
1.3882783882783958
0.0
0.0
0.0
0.0
0.0
10.83882783882784
4.67

### References
Jiang, Yi-qi, and Jian-ping Lin. "Recent progress in strategies for steroid production in yeasts." World Journal of Microbiology and Biotechnology 38.6 (2022): 1-14.